In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q transformers langchain llama-index llama-index-embeddings-huggingface
!pip install -q pypdf sentence_transformers accelerate bitsandbytes
!pip install llama-index transformers torch accelerate bitsandbytes
!pip install --upgrade llama-index llama-index-llms-huggingface transformers torch accelerate bitsandbytes



In [3]:
# Import required libraries
from llama_index.readers.file import PDFReader

# Load the PDF
pdf_path = "/content/drive/MyDrive/Colab Notebooks/edit2016.pdf"
documents = PDFReader().load_data(file=pdf_path)

# Check if the document loaded correctly
print(documents[0].text[:500])  # Print first 500 characters


 
BIO-MEDICAL WASTE MANAGEMENT RULES, 2016 as amended till 2019  
[Published in the Gazette of India, Extraordinary, Part II, Section 3, Sub-section (i)]  
 GOVERNMENT OF INDIA  
MINISTRY OF ENVIRONMENT, FOREST AND CLIMATE CHANGE  
NOTIFICATION  
New Delhi, the 28th March, 2016  
G.S.R. 343(E).-Whereas the Bio-Medical Waste (Management and Handling) Rules, 1998 was published 
vide notification number S.O. 630 (E) dated the 20 th July, 1998, by the Government of India in the 
erstwhile Ministry o


In [4]:
system_prompt = """
You are MedWaste Guardian, an AI assistant specializing in biomedical waste compliance.
Your goal is to provide accurate legal guidance based on the Bio-Medical Waste Management Rules, 2016.
Use retrieved legal texts to generate well-structured and compliant responses.
If information is unavailable, state so clearly.
"""
from llama_index.core.prompts.prompts import SimpleInputPrompt

# Define query wrapper prompt
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")


In [5]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-c

In [ ]:
# Install necessary dependencies (if not installed)
!pip install --upgrade llama-index llama-index-llms-huggingface transformers torch accelerate bitsandbytes

from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import BitsAndBytesConfig
import torch

# Configure quantization to optimize memory usage
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True  # Offload some layers to CPU if needed
)

# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
# Initialize the LLaMA-2 model
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 1.0, "do_sample": False},  # ✅ Fixed temperature issue
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",  # Auto-assigns layers across GPU/CPU
    model_kwargs={"torch_dtype": torch.float16, "quantization_config": bnb_config}
)

print("LLaMA-2 model initialized successfully!")


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Initialize embedding model and ensure it runs on GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2", device=device)

print("Embedding model loaded successfully on:", device)


In [ ]:
from huggingface_hub import login
from transformers import AutoTokenizer
from llama_index.core import Settings
from llama_index.llms.huggingface import HuggingFaceLLM


# ✅ Load tokenizer for LLaMA-2
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)  # Updated argument

# ✅ Ensure LlamaIndex doesn't default to OpenAI
Settings.llm = None

# ✅ Load LLaMA-2 model
llm = HuggingFaceLLM(model_name=model_name, tokenizer=tokenizer)

# ✅ Set LLaMA-2 as the LLM in LlamaIndex
Settings.llm = llm

print("✅ LLaMA-2 is now the active LLM:", Settings.llm.model_name)  # Avoid recursion error


In [ ]:
from llama_index.readers.file import PDFReader
from llama_index.core import VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# ✅ Use local embeddings instead of OpenAI
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.embed_model = embed_model  # Ensure LlamaIndex uses this embedding model

# ✅ Load PDF
pdf_path = "/content/drive/MyDrive/Colab Notebooks/edit2016.pdf"
documents = PDFReader().load_data(file=pdf_path)

# ✅ Create the index
index = VectorStoreIndex.from_documents(documents, settings=Settings)

print("✅ Index created successfully!")


In [ ]:

# Create a query engine from the index
query_engine = index.as_query_engine(llm=llm)


In [ ]:
# Example query
response = query_engine.query("How to dispose of syringes?")

# Print response
print(response)


In [ ]:
index.storage_context.persist(persist_dir="./medwaste_index")


In [ ]:
!zip -r /content/medwaste_guardian_rag.zip /content/drive/MyDrive/Colab\ Notebooks/medwaste_index


In [ ]:
from google.colab import files
files.download("/content/medwaste_guardian_rag.zip")


In [ ]:
#pip install pypdf
#pip install -q transformers einops accelerate langchain bitsandbytes
#pip install llama_index
#pip install --upgrade llama-index llama-index-llms-huggingface transformers
#pip install llama-index llama-index-readers-file
#pip install sentence_transformers
#pip install -U llama-index-embeddings-huggingface langchain langchain-community
#pip install fastapi uvicorn


In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM
llm = HuggingFaceLLM(model_name="meta-llama/Llama-2-7b-chat-hf")
print("LLaMA-2 model loaded successfully!")


In [ ]:
response = query_engine.query("What are the biomedical waste disposal rules for human anatomical waste?")
print(response)
